In [2]:
import os, shutil


In [3]:
original_dataset_dir='./train'

base_dir = './data/cats_and_dogs'

if not os.path.exists(base_dir):
    os.mkdir(base_dir)

In [6]:
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

In [7]:
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)


In [8]:
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)


In [10]:
fnames = ['cat.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
    

In [13]:
fnames = ['cat.{}.jpg'.format(i) for i in range(10000, 12500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

In [12]:
fnames = ['dog.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    

In [14]:
fnames = ['dog.{}.jpg'.format(i) for i in range(10000, 12500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [15]:
print('훈련용 고양이 이미지 전체 개수:', len(os.listdir(train_cats_dir)))
print('훈련용 강아지 이미지 전체 개수:', len(os.listdir(train_dogs_dir)))
print('테스트용 고양이 이미지 전체 개수:', len(os.listdir(test_cats_dir)))
print('테스트용 강아지 이미지 전체 개수:', len(os.listdir(test_dogs_dir)))

훈련용 고양이 이미지 전체 개수: 10000
훈련용 강아지 이미지 전체 개수: 10000
테스트용 고양이 이미지 전체 개수: 2500
테스트용 강아지 이미지 전체 개수: 2500


In [16]:
from tensorflow.keras import models, layers
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# 여러개의 커널은 해당 이미지 분석을 다양하게 하기 위해서 여러개가 존재함
# 층은 적당히 조절하는게 좋다

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [32]:
32*3*9+32

896

In [40]:
32*64*9 + 64

18496

In [42]:
128*64*9 + 128

73856

In [44]:
128*128*9 + 128

147584

In [47]:
7*7*512*128+512

3211776

In [49]:
896+18496+73856+147584+3211776+513

3453121

In [52]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale= 1./255)
test_datagen = ImageDataGenerator(rescale= 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150), # 파일사이즈를 조정함
    batch_size=20,
    class_mode='binary' # 폴더내의 target name을 0, 1로 분류, 다중분류는 categorical
    )

vaildation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
    )

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [56]:
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=vaildation_generator
)

Epoch 1/30
1000/1000 [==============================] - 425s 424ms/step - loss: 0.6800 - accuracy: 0.5612 - val_loss: 0.6464 - val_accuracy: 0.6072
Epoch 2/30
1000/1000 [==============================] - 232s 232ms/step - loss: 0.5834 - accuracy: 0.6860 - val_loss: 0.5198 - val_accuracy: 0.7438
Epoch 3/30
1000/1000 [==============================] - 230s 230ms/step - loss: 0.4938 - accuracy: 0.7559 - val_loss: 0.4468 - val_accuracy: 0.7918
Epoch 4/30
1000/1000 [==============================] - 229s 229ms/step - loss: 0.4260 - accuracy: 0.8039 - val_loss: 0.4295 - val_accuracy: 0.7996
Epoch 5/30
1000/1000 [==============================] - 230s 230ms/step - loss: 0.3619 - accuracy: 0.8397 - val_loss: 0.3595 - val_accuracy: 0.8422
Epoch 6/30
1000/1000 [==============================] - 231s 230ms/step - loss: 0.3173 - accuracy: 0.8629 - val_loss: 0.3645 - val_accuracy: 0.8400
Epoch 7/30
1000/1000 [==============================] - 238s 238ms/step - loss: 0.2767 - accuracy: 0.8807 - val_

In [58]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

AttributeError: 'function' object has no attribute 'history'